# Three kinds of Implicits and Type Classes

blah blah blah

### -- Implicit Arguments --

Suppose you want a function that joins two strings together.

In [1]:
def example1(a: String, b: String): String = List(a, b).mkString("")

example1("foo", "bar")

defined function example1
res0_1: String = "foobar"

You can split the arguments into a parameter list as such.

In [6]:
def example2(a: String)(b: String): String = List(a, b).mkString("")

example2("foo")("bar")

defined function example2
res5_1: String = "foobar"
res5_2: String = "foobar"

blah blah blah

In [7]:
val partial = example2("foo")(_)
partial("bar")

partial: String => String = ammonite.$sess.cmd6$Helper$$Lambda$2702/227216095@624a9ccd
res6_1: String = "foobar"

blah blah blah

In [3]:
def example3(a: String)(implicit b: String): String = List(a, b).mkString("")

implicit val bar: String = "bar"

example3("foo")

defined function example3
bar: String = "bar"
res2_2: String = "foobar"

blah blah blah

### -- Implicit Type Conversion --

In [5]:
object Example4 {
  implicit def convert(x: Example5): Example4 = {
    val Array(firstName, lastName) = x.fullName.split(" ")
    Example4(firstName, lastName)
  }
}

case class Example4(firstName: String, lastName: String)

object Example5 {
  implicit def convert(x: Example4): Example5 = {
    val fullName = List(x.firstName, x.lastName).mkString(" ")
    Example5(fullName)
  }
}

case class Example5(fullName: String)

val a: Example4 = Example4("Bob", "Smith")
val b: Example5 = a
val c: Example4 = b

defined object Example4
defined class Example4
defined object Example5
defined class Example5
a: Example4 = Example4("Bob", "Smith")
b: Example5 = Example5("Bob Smith")
c: Example4 = Example4("Bob", "Smith")

### -- Implicit Method --

blah blah blah